In [1]:
import sys
sys.path.append('../30_data_tools/')

In [14]:
import numpy as np
from scipy.ndimage import gaussian_filter
from pathlib import Path
from PIL import Image
import torchvision
import torch
from helper import load_dotenv
import plotly.express as px
import math
from random import choices
from tqdm.auto import tqdm

In [3]:
dotenv = load_dotenv()

In [4]:
tile_paths = list((dotenv['TILE_DATASET_DIR'] / 'train' / 'moire').glob('./*.jpg'))

In [5]:
R_MAP = torch.zeros((224,224))

for y in range(224):
    R_MAP[y,:] = torch.hypot(torch.arange(224) - 112, torch.Tensor([y - 112]))

In [24]:
def get_fft( input_img ):
    ft = np.fft.ifftshift(np.array(input_img))
    ft = np.fft.fft2(ft)
    ft = np.fft.fftshift(ft)
    
    return ft

def get_fft_torch( input_img ):
    img_torch = torchvision.transforms.functional.to_tensor(input_img).squeeze(0) * 255
    ft = torch.fft.ifftshift( img_torch )
    ft = torch.fft.fft2( ft )
    ft = torch.fft.fftshift( ft )

    return ft

In [7]:
def limit_frequencies( fft, inner_limit=None, outer_limit=None ):
    center = (fft.shape[1] / 2, fft.shape[0] / 2)
    for y in range(fft.shape[0]):
        for x in range(fft.shape[1]):
            r = math.sqrt( abs(center[0] - x) ** 2 + abs(center[1] - y) ** 2 )
            
            if outer_limit is not None and r > outer_limit:
                fft[y,x] = 1
    
            if inner_limit is not None and r < inner_limit:
                fft[y,x] = 1

    return fft


def limit_frequencies_torch( fft, inner_limit=None, outer_limit=None ):
    if (inner_limit is None) == False:
        fft[R_MAP <= inner_limit] = 1

    if (outer_limit is None) == False:
        fft[R_MAP >= outer_limit] = 1
    
    return fft

In [8]:
def get_frequency_representation( img ):
    fft = np.abs( limit_frequencies( get_fft(img), inner_limit=5 ) )
    fft = gaussian_filter(fft, sigma=3)

    return fft


def get_frequency_representation_torch( img ):
    fft = torch.abs( limit_frequencies_torch( get_fft_torch(img), inner_limit=5 ) ).reshape((1,224,224))    
    fft = torchvision.transforms.functional.gaussian_blur(fft, (11,11), sigma=3)[0,:,:]

    return fft

In [20]:
def img_to_fft( img ):
    fft = get_frequency_representation(img.convert('L'))
    channel = (fft / fft.max() * 255).astype('uint8')
    fft = np.zeros((channel.shape[0],channel.shape[1],3)).astype('uint8')
    fft[:,:,0] = channel
    fft[:,:,1] = channel
    fft[:,:,2] = channel

    return fft


def img_to_fft_torch( img ):
    fft = get_frequency_representation_torch(img.convert('L'))
    channel = (fft / fft.max() * 255)
    fft = torch.zeros((channel.shape[0],channel.shape[1],3))
    fft[:,:,0] = channel
    fft[:,:,1] = channel
    fft[:,:,2] = channel

    return fft

In [32]:
transformation = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [33]:
transformation(tile_img)

tensor([[[0.7882, 0.7059, 0.7843,  ..., 0.0667, 0.0706, 0.0510],
         [0.7020, 0.7843, 0.7333,  ..., 0.0471, 0.0471, 0.0314],
         [0.7922, 0.7176, 0.7961,  ..., 0.0510, 0.0549, 0.0471],
         ...,
         [0.7804, 0.8667, 0.7529,  ..., 0.8471, 0.8471, 0.8627],
         [0.7255, 0.7216, 0.7686,  ..., 0.7922, 0.8824, 0.8353],
         [0.8000, 0.8510, 0.7529,  ..., 0.9255, 0.8157, 0.8627]],

        [[0.7882, 0.7059, 0.7843,  ..., 0.0667, 0.0706, 0.0510],
         [0.7020, 0.7843, 0.7333,  ..., 0.0471, 0.0471, 0.0314],
         [0.7922, 0.7176, 0.7961,  ..., 0.0510, 0.0549, 0.0471],
         ...,
         [0.7804, 0.8667, 0.7529,  ..., 0.8471, 0.8471, 0.8627],
         [0.7255, 0.7216, 0.7686,  ..., 0.7922, 0.8824, 0.8353],
         [0.8000, 0.8510, 0.7529,  ..., 0.9255, 0.8157, 0.8627]],

        [[0.7882, 0.7059, 0.7843,  ..., 0.0667, 0.0706, 0.0510],
         [0.7020, 0.7843, 0.7333,  ..., 0.0471, 0.0471, 0.0314],
         [0.7922, 0.7176, 0.7961,  ..., 0.0510, 0.0549, 0.

In [11]:
test_selection = choices( tile_paths, k=100 )

In [17]:
to_tensor = torchvision.transforms.ToTensor()

In [25]:
for tile_path in test_selection:
    tile_img = Image.open( tile_path )
    torch_fft = img_to_fft_torch( tile_img )    
    numpy_fft = img_to_fft( tile_img )

    print( (torch_fft - numpy_fft).mean() )

tensor(-1.1221)
tensor(-0.1420)
tensor(0.4367)
tensor(-0.1077)
tensor(0.7928)
tensor(-0.9919)
tensor(-1.1413)
tensor(-0.8876)
tensor(-1.2091)
tensor(-1.6106)
tensor(-0.7549)
tensor(-1.0580)
tensor(0.0373)
tensor(-1.2519)
tensor(0.2472)
tensor(-1.0200)
tensor(-0.2944)
tensor(-0.9336)
tensor(-1.5914)
tensor(-0.5373)
tensor(-1.1958)
tensor(-1.2909)
tensor(0.3818)
tensor(-0.1230)
tensor(0.3658)
tensor(0.0854)
tensor(0.3932)
tensor(-1.2858)
tensor(-0.8522)
tensor(-0.2268)
tensor(-0.9675)
tensor(-0.7761)
tensor(-0.0938)
tensor(-0.3343)
tensor(-0.6338)
tensor(-0.7156)
tensor(-0.3466)
tensor(-0.5789)
tensor(-0.0419)
tensor(-1.1488)
tensor(-0.0120)
tensor(-0.0454)
tensor(-0.5175)
tensor(-0.3274)
tensor(-0.7052)
tensor(-0.5568)
tensor(-1.4842)
tensor(0.7021)
tensor(-2.0768)
tensor(0.0678)
tensor(-1.0691)
tensor(-0.7315)
tensor(-0.7802)
tensor(-1.6545)
tensor(-0.0579)
tensor(-0.5366)
tensor(-0.5989)
tensor(-1.1126)
tensor(-0.8457)
tensor(-1.6733)
tensor(0.0210)
tensor(-1.8718)
tensor(0.1032)
tens

KeyboardInterrupt: 

In [ ]:
torch.abs(torch_fft - numpy_fft).mean()

In [ ]:
px.imshow(test)

In [ ]:
np.array(tile_img.convert('L'))

In [ ]:
img_torch = to_tensor(tile_img.convert('L')).squeeze(0) * 255
fft = torch.fft.ifftshift( img_torch )
fft = torch.fft.fft2( fft )
fft = torch.fft.fftshift( fft )

In [ ]:
fft[:5,:5].real - get_fft( tile_img.convert('L') )[:5,:5].real

In [ ]:
fft[:5,:5].real

In [ ]:
get_fft( tile_img.convert('L') )[:5,:5].real